In [1]:
from graph.process.nodes import make_execution_agent
config = {"provider":"anthropic","model_type":"slow"}
config

{'provider': 'anthropic', 'model_type': 'slow'}

In [2]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

# 1. Import your Node Factory
# (Make sure your imports in the files are correct first)
from graph.process.nodes.execution import make_execution_agent
from graph.process.schemas import ProcessPlan, PlanStep, MetricSpec

# ---------------------------------------------------------
# 2. SETUP MOCK DATA
# ---------------------------------------------------------

# Create a mock Plan (what the Planner would have produced)
mock_plan = ProcessPlan(
    needs_clarification=False,
    response_type="DATA_LOOKUP",
    selected_sources=["USER_METRICS", "KNOWLEDGE_BASE"],
    confidence=0.9,
    metrics=[
        MetricSpec(name="steps_daily", aggregation="avg"),
        MetricSpec(name="heart_rate", aggregation="avg")
    ],
    steps=[
        PlanStep(
            step_id="1",
            action="fetch_knowledge_base",
            notes="Check standard water intake guidelines."
        ),
        PlanStep(
            step_id="2",
            action="synthesize_answer",
            notes="Combine guidelines with user context."
        )
    ]
)

# Create a mock State (what the Graph passes to the node)
mock_state = {
    "messages": [
        HumanMessage(content="How much water should I drink daily?"),
    ],
    "process_plan": mock_plan,
    "user_profile": None, # Optional: Add mock profile if needed
}

# ---------------------------------------------------------
# 3. INSTANTIATE AND RUN
# ---------------------------------------------------------

# Initialize the Node with config
# 'provider' can be 'ollama', 'anthropic', or 'openai' depending on your setup
config = {"provider": "ollama", "model_type": "slow"}

print(f"--- Building Agent ({config['provider']}) ---")
execution_node = make_execution_agent(config)

print("--- Executing Node (This may take a moment) ---")
try:
    result = execution_node(mock_state)

    # ---------------------------------------------------------
    # 4. EVALUATE OUTPUT
    # ---------------------------------------------------------
    print("\n✅ EXECUTION COMPLETE\n")

    # Check the actual text response
    final_message = result["messages"][-1]
    print(f"🔹 Final Answer:\n{final_message.content}\n")

    # Check the metadata extracted from the structured output
    print(f"🔹 Confidence: {result.get('grounding_metadata', {}).get('confidence')}")
    print(f"🔹 Needs Clarification: {result.get('needs_clarification')}")

    # Check for raw response storage
    if "response" in result:
        print(f"🔹 Stored Response: {result['response'][:50]}...")

except Exception as e:
    print(f"\n❌ EXECUTION FAILED: {e}")
    import traceback
    traceback.print_exc()

--- Building Agent (ollama) ---
--- Executing Node (This may take a moment) ---
---RETRIEVE---
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GENERATE---

✅ EXECUTION COMPLETE

🔹 Final Answer:
Based on standard guidelines, the recommended daily water intake for adults is approximately **3.7 liters (15.5 cups) for men** and **2.7 liters (11.5 cups) for women**. Since your average daily step count is around **12,350 steps**, which indicates moderate activity, you may need to drink slightly more water than the baseline recommendation to stay hydrated. If you engage in intense exercise, live in a hot climate, or have specific health conditions, your water needs could be higher. Always listen to your body and adjust accordingly.

*Note: No heart rate data was available for this period, so adjustments based on cardiovascular activity could not be made.*

🔹 Confid